In [1]:
#import necessary library
import pandas as pd
import requests
import csv
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup #For data scrapping

In [2]:
#For mounting the drive
#If you want to run it on your local machine then skip it and change the csv diractory as your own interest
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
#Html Request to the website.
page = requests.get("https://www.worldometers.info/coronavirus/?fbclid=IwAR0L9-HT-jDr8yXdZaeIsKT_kLkwFyGxwGGa8ezWQ2lGRrDd4Kjk_zqWtE8#news")
soup = BeautifulSoup(page.content,'html.parser')
#from the website, All we need is the table.
table = soup.findAll("table", {"id":"main_table_countries_yesterday"})[0]
# Fatch all the rows and store in a list.
rows = table.findAll("tr")

# Store as a CSV format

with open("/content/gdrive/My Drive/Colab Notebooks/COVID-19/covid_update2.csv", "wt+",encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    for row in rows:
      csv_row = []
      for cell in row.findAll(["td", "th"]):
        csv_row.append(cell.get_text())
      writer.writerow(csv_row)


# Data Exploration

In [4]:
#importing necessary library
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set() # Apply the default seaborn theme, scaling, and color palette

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [20]:
df1 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/COVID-19/covid_update.csv")
df2 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/COVID-19/covid_update2.csv")

In [21]:
cols1 = df1.columns
cols2 = df2.columns

In [22]:
flag = 0
for col in cols1:
  if col in cols2:
    df1[col] = df2[col]
df = df1


In [9]:
df.head(10)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,NaN,\nAsia\n,"2,959,505","+56,719","70,286","+1,349","2,046,345","+44,413","842,874","19,731",NaN,NaN,NaN,NaN,NaN,Asia,\n,NaN,NaN
1,NaN,\nNorth America\n,"3,995,688","+69,432","185,804","+1,042","1,837,700","+33,300","1,972,184","19,168",NaN,NaN,NaN,NaN,NaN,North America,\n,NaN,NaN
2,NaN,\nSouth America\n,"2,893,624","+42,450","105,156","+1,242","1,873,771","+9,923","914,697","13,691",NaN,NaN,NaN,NaN,NaN,South America,\n,NaN,NaN
3,NaN,\nEurope\n,"2,570,985","+11,292","196,566",+228,"1,511,540","+6,716","862,879","5,345",NaN,NaN,NaN,NaN,NaN,Europe,\n,NaN,NaN
4,NaN,\nAfrica\n,"598,236","+16,596","13,285",+256,"298,194","+10,270","286,757",969,NaN,NaN,NaN,NaN,NaN,Africa,\n,NaN,NaN
5,NaN,\nOceania\n,"11,460",+244,130,+1,"9,331",NaN,"1,999",17,NaN,NaN,NaN,NaN,NaN,Australia/Oceania,\n,NaN,NaN
6,NaN,\n\n,721,NaN,15,NaN,651,NaN,55,4,NaN,NaN,NaN,NaN,NaN,NaN,\n,NaN,NaN
7,NaN,World,"13,030,219","+196,733","571,242","+4,118","7,577,532","+104,619","4,881,445","58,925","1,672",73.3,NaN,NaN,NaN,All,\n,NaN,NaN
8,1.0,China,"83,594",+7,"4,634",NaN,"78,634",+11,326,3,58,3,"90,410,000","62,814","1,439,323,776",Asia,"17,218","310,601",16
9,2.0,USA,"3,413,995","+58,349","137,782",+380,"1,517,084","+27,664","1,759,129","15,822","10,312",416,"42,469,607","128,281","331,065,797",North America,97,"2,403",8


At first we need to name our features in a meaningful and authentic way. 

In [23]:
df.columns = ['#','Country','TotalCases','NewCases','TotalDeaths','NewDeaths','TotalRecovered','NewRecovered','ActiveCases','SeriousCritical','TotalCasesPerMillion','DeathsPerMillion','TotalTests','TestsPerMillion','Population','Continent','CaseEvery','DeathEvery','TestEvery']
df.head()

,#,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent,CaseEvery,DeathEvery,TestEvery
0,NaN,\nAsia\n,"2,959,505","+56,719","70,286","+1,349","2,046,345","+44,413","842,874","19,731",NaN,NaN,NaN,NaN,NaN,Asia,\n,NaN,NaN
1,NaN,\nNorth America\n,"3,995,688","+69,432","185,804","+1,042","1,837,700","+33,300","1,972,184","19,168",NaN,NaN,NaN,NaN,NaN,North America,\n,NaN,NaN
2,NaN,\nSouth America\n,"2,893,624","+42,450","105,156","+1,242","1,873,771","+9,923","914,697","13,691",NaN,NaN,NaN,NaN,NaN,South America,\n,NaN,NaN
3,NaN,\nEurope\n,"2,570,985","+11,292","196,566",+228,"1,511,540","+6,716","862,879","5,345",NaN,NaN,NaN,NaN,NaN,Europe,\n,NaN,NaN
4,NaN,\nAfrica\n,"598,236","+16,596","13,285",+256,"298,194","+10,270","286,757",969,NaN,NaN,NaN,NaN,NaN,Africa,\n,NaN,NaN


We've seen some columns are not that much rellivent. We'll reduce those columns

In [24]:
df = df.drop(['#','CaseEvery','DeathEvery','TestEvery'], axis=1)
df.head(10)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
0,\nAsia\n,"2,959,505","+56,719","70,286","+1,349","2,046,345","+44,413","842,874","19,731",NaN,NaN,NaN,NaN,NaN,Asia
1,\nNorth America\n,"3,995,688","+69,432","185,804","+1,042","1,837,700","+33,300","1,972,184","19,168",NaN,NaN,NaN,NaN,NaN,North America
2,\nSouth America\n,"2,893,624","+42,450","105,156","+1,242","1,873,771","+9,923","914,697","13,691",NaN,NaN,NaN,NaN,NaN,South America
3,\nEurope\n,"2,570,985","+11,292","196,566",+228,"1,511,540","+6,716","862,879","5,345",NaN,NaN,NaN,NaN,NaN,Europe
4,\nAfrica\n,"598,236","+16,596","13,285",+256,"298,194","+10,270","286,757",969,NaN,NaN,NaN,NaN,NaN,Africa
5,\nOceania\n,"11,460",+244,130,+1,"9,331",NaN,"1,999",17,NaN,NaN,NaN,NaN,NaN,Australia/Oceania
6,\n\n,721,NaN,15,NaN,651,NaN,55,4,NaN,NaN,NaN,NaN,NaN,NaN
7,World,"13,030,219","+196,733","571,242","+4,118","7,577,532","+104,619","4,881,445","58,925","1,672",73.3,NaN,NaN,NaN,All
8,China,"83,594",+7,"4,634",NaN,"78,634",+11,326,3,58,3,"90,410,000","62,814","1,439,323,776",Asia
9,USA,"3,413,995","+58,349","137,782",+380,"1,517,084","+27,664","1,759,129","15,822","10,312",416,"42,469,607","128,281","331,065,797",North America


The First 8 Rows data is continantal data. We will deal with only countries data. So We're going to delete first 8 row from this dataset. 

In [25]:
df = df.iloc[8:]
df.head()


,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
8,China,"83,594",+7,"4,634",NaN,"78,634",+11,326,3,58,3,"90,410,000","62,814","1,439,323,776",Asia
9,USA,"3,413,995","+58,349","137,782",+380,"1,517,084","+27,664","1,759,129","15,822","10,312",416,"42,469,607","128,281","331,065,797",North America
10,Brazil,"1,866,176","+25,364","72,151",+659,"1,213,512",NaN,"580,513","8,318","8,778",339,"4,572,796","21,508","212,607,642",South America
11,India,"879,466","+29,108","23,187",+500,"554,429","+18,198","301,850","8,944",637,17,"11,587,153","8,394","1,380,418,989",Asia
12,Russia,"727,162","+6,615","11,335",+130,"501,061","+3,615","214,766","2,300","4,983",78,"23,031,056","157,815","145,936,664",Europe


We can see the index start from 8. So we're going to reset the index number.

In [26]:
df = df.reset_index(drop=True)
df.head()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
0,China,"83,594",+7,"4,634",NaN,"78,634",+11,326,3,58,3,"90,410,000","62,814","1,439,323,776",Asia
1,USA,"3,413,995","+58,349","137,782",+380,"1,517,084","+27,664","1,759,129","15,822","10,312",416,"42,469,607","128,281","331,065,797",North America
2,Brazil,"1,866,176","+25,364","72,151",+659,"1,213,512",NaN,"580,513","8,318","8,778",339,"4,572,796","21,508","212,607,642",South America
3,India,"879,466","+29,108","23,187",+500,"554,429","+18,198","301,850","8,944",637,17,"11,587,153","8,394","1,380,418,989",Asia
4,Russia,"727,162","+6,615","11,335",+130,"501,061","+3,615","214,766","2,300","4,983",78,"23,031,056","157,815","145,936,664",Europe


In [14]:
#Data discription
df.describe()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
count,223,223,144,223,88,218,116,219,136,214,186,194,194,215,220
unique,216,221,114,145,40,208,96,182,77,204,104,193,194,214,7
top,Total:,12,+3,,+1,19,+1,0,1,687,3,"275,000","128,281",,Africa
freq,8,2,5,28,15,2,5,18,17,2,11,2,1,2,58


From this Description We got some interesting instinct. <br>
### 1: Tatal affected countries = 215 
### 2: Most affectected Continent = Africa
### 3: Total affect countries in Africa = 58

In [15]:
df.shape

(223, 15)

In [16]:
df.tail(10)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
213,Anguilla,3,NaN,,NaN,3,NaN,0,NaN,200,NaN,380,"25,322","15,007",North America
214,Saint Pierre Miquelon,2,NaN,,NaN,1,NaN,1,NaN,345,NaN,NaN,NaN,"5,793",North America
215,Total:,"2,959,505","+56,719","70,286","+1,349","2,046,345","+44,413","842,874","19,731",NaN,NaN,NaN,NaN,NaN,Asia
216,Total:,"3,995,688","+69,432","185,804","+1,042","1,837,700","+33,300","1,972,184","19,168",NaN,NaN,NaN,NaN,NaN,North America
217,Total:,"2,893,624","+42,450","105,156","+1,242","1,873,771","+9,923","914,697","13,691",NaN,NaN,NaN,NaN,NaN,South America
218,Total:,"2,570,985","+11,292","196,566",+228,"1,511,540","+6,716","862,879","5,345",NaN,NaN,NaN,NaN,NaN,Europe
219,Total:,"598,236","+16,596","13,285",+256,"298,194","+10,270","286,757",969,NaN,NaN,NaN,NaN,NaN,Africa
220,Total:,"11,460",+244,130,+1,"9,331",+-3,"1,999",17,NaN,NaN,NaN,NaN,NaN,Australia/Oceania
221,Total:,721,NaN,15,NaN,651,NaN,55,4,NaN,NaN,NaN,NaN,NaN,NaN
222,Total:,"13,030,219","+196,733","571,242","+4,118","7,577,532","+104,619","4,881,445","58,925","1,671.7",73.3,NaN,NaN,NaN,All


In [27]:
row_size, col_size = df.shape

In [28]:

df = df.drop(df.index[row_size-8:row_size-1])
df = df.reset_index(drop=True)
df.tail()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
211,Caribbean Netherlands,7,NaN,,NaN,7,NaN,0,NaN,267,NaN,424,"16,164","26,231",North America
212,St. Barth,6,NaN,,NaN,6,NaN,0,NaN,607,NaN,507,"51,326","9,878",North America
213,Anguilla,3,NaN,,NaN,3,NaN,0,NaN,200,NaN,380,"25,322","15,007",North America
214,Saint Pierre Miquelon,2,NaN,,NaN,1,NaN,1,NaN,345,NaN,NaN,NaN,"5,793",North America
215,Total:,"13,030,219","+196,733","571,242","+4,118","7,577,532","+104,619","4,881,445","58,925","1,671.7",73.3,NaN,NaN,NaN,All


In [30]:
df.fillna(0, inplace=True)
df.tail()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
211,Caribbean Netherlands,7,0,,0,7,0,0,0,267,0,424,"16,164","26,231",North America
212,St. Barth,6,0,,0,6,0,0,0,607,0,507,"51,326","9,878",North America
213,Anguilla,3,0,,0,3,0,0,0,200,0,380,"25,322","15,007",North America
214,Saint Pierre Miquelon,2,0,,0,1,0,1,0,345,0,0,0,"5,793",North America
215,Total:,"13,030,219","+196,733","571,242","+4,118","7,577,532","+104,619","4,881,445","58,925","1,671.7",73.3,0,0,0,All


In [44]:
total_null = df[df.Continent==0].index.tolist() + df[df.Country==0].index.tolist()
total_null = list(set(total_null))
df.drop(df.index[total_null], inplace=True)
df.shape

(214, 15)

In [53]:
df = df[df.columns].replace({'\+':''}, regex = True)
df.tail()



,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
211,Caribbean Netherlands,7,0,,0,7,0,0,0,267,0,424,"16,164","26,231",North America
212,St. Barth,6,0,,0,6,0,0,0,607,0,507,"51,326","9,878",North America
213,Anguilla,3,0,,0,3,0,0,0,200,0,380,"25,322","15,007",North America
214,Saint Pierre Miquelon,2,0,,0,1,0,1,0,345,0,0,0,"5,793",North America
215,Total:,"13,030,219","196,733","571,242","4,118","7,577,532","104,619","4,881,445","58,925","1,671.7",73.3,0,0,0,All


In [54]:
df = df.replace(' ', 0)
df.tail()
                           
                           

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
211,Caribbean Netherlands,7,0,0,0,7,0,0,0,267,0,424,"16,164","26,231",North America
212,St. Barth,6,0,0,0,6,0,0,0,607,0,507,"51,326","9,878",North America
213,Anguilla,3,0,0,0,3,0,0,0,200,0,380,"25,322","15,007",North America
214,Saint Pierre Miquelon,2,0,0,0,1,0,1,0,345,0,0,0,"5,793",North America
215,Total:,"13,030,219","196,733","571,242","4,118","7,577,532","104,619","4,881,445","58,925","1,671.7",73.3,0,0,0,All
